In [ ]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just
%install '.package(url: "https://github.com/twostraws/SwiftGD.git", from: "2.0.0")' SwiftGD

In [ ]:
import Foundation
import Path
import Just

In [ ]:
//export
public func shell_cmd(_ launchPath: String, _ arguments: [String]) -> String?
{
    let task = Process()
    task.executableURL = URL.init(fileURLWithPath:launchPath)
    task.arguments = arguments

    let pipe = Pipe()
    task.standardOutput = pipe
    do {try task.run()} catch {print("Unexpected error: \(error).")}

    let data = pipe.fileHandleForReading.readDataToEndOfFile()
    let output = String(data: data, encoding: String.Encoding.utf8)

    return output
}

In [ ]:
if let res = shell_cmd("/bin/ls", ["-lh"]){print(res)}

In [ ]:
//export
public func download_file(_ url: String, dest: String?=nil, force: Bool=false){
    let dest_name = (dest ?? (Path.cwd/url.split(separator: "/").last!).string)
    let url_dest = URL.init(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if (force || !Path(dest_name)!.exists){
        print("Downloading \(url)...")
        if let cts = Just.get(url).content{
            do    {try cts.write(to: URL.init(fileURLWithPath:dest_name))}
            catch {print("Can't write to \(url_dest).\n\(error)")}
        } else {print("Can't reach \(url)")}
    }
}

In [ ]:
download_file("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")

In [ ]:
//export
import TensorFlow

In [ ]:
//export
protocol ConvertableFromByte {
    init(_ d:UInt8)
}

In [ ]:
//export
extension Float : ConvertableFromByte{}
extension Int32 : ConvertableFromByte{}

In [ ]:
//export
func get_data<T:ConvertableFromByte & TensorFlowScalar>(_ fn:String, _ skip:Int) -> Tensor<T> {
    let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: fn)).dropFirst(skip)
    return Tensor(data.map(T.init))
}

In [ ]:
// export
public struct MnistDataset{
    let base_url = "http://yann.lecun.com/exdb/mnist/"
    let trn_imgs = "train-images-idx3-ubyte"
    let trn_lbls = "train-labels-idx1-ubyte"
    let val_imgs = "t10k-images-idx3-ubyte"
    let val_lbls = "t10k-labels-idx1-ubyte" 
    
    var path = Path.cwd
    
    public init(path: Path){
        self.path = path
        if !path.exists {try! path.mkdir()}
        let data_files = [path/trn_imgs, path/trn_lbls, path/val_imgs, path/val_lbls]
        for file in data_files{
            if !file.exists {
                let fname = file.basename()
                download_file("\(base_url)\(fname).gz", dest:(path/"\(fname).gz").string)
                _ = shell_cmd("/bin/gunzip", ["-fq", (path/"\(fname).gz").string])
            }
        }
    }
    
    func get_data<T:ConvertableFromByte & TensorFlowScalar>(_ fn:String, _ skip:Int) -> Tensor<T> {
        let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: fn)).dropFirst(skip)
        return Tensor(data.map(T.init))
    }
    
    public var xTrain: Tensor<Float> {return get_data((path/trn_imgs).string, 16)/255.0}
    public var yTrain: Tensor<Int32> {return get_data((path/trn_lbls).string, 8)}
    public var xValid: Tensor<Float> {return get_data((path/val_imgs).string, 16)/255.0}
    public var yValid: Tensor<Int32> {return get_data((path/val_lbls).string, 8)}
}

In [ ]:
let mnist = MnistDataset(path: Path.cwd/"data"/"mnist_tst")

In [ ]:
//export 
import Dispatch
public func time(_ function: () -> ()) {
    let start = DispatchTime.now()
    function()
    let end = DispatchTime.now()
    let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    let milliseconds = nanoseconds / 1e6
    print("\(milliseconds) ms")
}

In [ ]:
time {var trn_imgs = mnist.xTrain}

In [ ]:
//export 
public func time(repeating: Int, _ function: () -> ()) {
    var times:[Double] = []
    for _ in 1...repeating{
        let start = DispatchTime.now()
        function()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("\(times.reduce(0.0, +)/Double(times.count)) ms")
}

In [ ]:
time(repeating:10) {var trn_imgs = mnist.xTrain}

## Download Imagenette

https://github.com/fastai/imagenette/

In [ ]:
let path = Path.cwd/"data"/"imagenette"
let base_url = "https://s3.amazonaws.com/fast-ai-imageclas/"
let imagenette_small = "imagenette-160.tgz"

let downloaded = path/imagenette_small

if !downloaded.exists {
    download_file("\(base_url)\(imagenette_small)", dest:(path/imagenette_small).string)
    
    _ = shell_cmd("/bin/tar", ["-xzf", (path/imagenette_small).string])
}

In [ ]:
// Prev cell untars file in .cwd, and it's a bit cumbersome to fix, so let that be for now

In [ ]:
let path = Path.cwd/"imagenette-160"/"train"

In [ ]:
let categories = path.ls().directories.map { $0.basename() } 

In [ ]:
print(categories)

In [ ]:
let imgs = categories.flatMap { try (path/$0).ls().files }

In [ ]:
let img = imgs[0]

In [ ]:
let fpath = URL(fileURLWithPath: img.string)

In [ ]:
import SwiftGD

In [ ]:
let im:Image = Image(url: fpath)!

In [ ]:
import Python

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

let plt = Python.import("matplotlib.pyplot")
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
let mpimg = Python.import("matplotlib.image")

In [ ]:
var pixels:[[[Double]]] = []

In [ ]:
let px = im.get(pixel: Point(x:0, y:0))

In [ ]:
print(px)

In [ ]:
let w = im.size.width
let h = im.size.height

In [ ]:
for i in 0...w {
    var row:[[Double]] = []
    for j in 0...h {
        row.append( im.get(pixel: Point(x:i, y:j)).asRGB )
    }
    pixels.append(row)
}

In [ ]:
pixels[239][159]

In [ ]:
extension Color {
    public func asRGB() -> [Double] {
        return [redComponent, greenComponent, blueComponent]
    }
}